## This notebook contains scripts that extract arthropod protein accessions in NR

In [23]:
%%bash
##extract arthropod taxids from ncbi nodes.dmp
taxid=6656                      # Arthropoda
nodes='/net/bos-nfsisilon/ifs/rc_labs/extavour_lab/rkapoor/dbs/tax_db/nodes.dmp'
out='outputs/arthropoda.taxids'

awk -F '\t\\|\t' -v root="$taxid" '
    NR==FNR { parent[$1] = $2; next }            # 1st pass
    function is_descendant(id) {
        while (id && id != 1 && id != root) id = parent[id];
        return id == root;
    }
    END {
        for (id in parent)
            if (is_descendant(id))
                print id;
    }
' "$nodes" "$nodes" > "$out"

In [25]:
%%bash
# Extract all arthropod protein accessions in NR, using  the extracted arthropoda.taxids
LC_ALL=C sort -u outputs/arthropoda.taxids > outputs/sorted_arthropoda.taxids

parallel --pipepart -a /net/bos-nfsisilon/ifs/rc_labs/extavour_lab/rkapoor/dbs/tax_db/accession2taxid.tsv \
        -j32 --block 2G \
        "awk -F'\t' 'NR==FNR {t[\$1]; next} (\$2 in t){print \$1}' \
             outputs/sorted_arthropoda.taxids -" \
    > outputs/arthropoda.accessions.raw
LC_ALL=C sort -u -T /scratch -S20G \
      outputs/arthropoda.accessions.raw \
      -o outputs/arthropoda.accessions